In [26]:
import soccerdata as sd
import pandas as pd
from tqdm import tqdm
import ScraperFC as sfc
import traceback

import warnings
import uuid

In [27]:
## Create a connection to Snowflake
from sqlalchemy import create_engine
import snowflake.connector

In [28]:
import snowflake.connector

In [29]:
from snowflake.connector.pandas_tools import write_pandas
from snowflake.sqlalchemy import URL

In [30]:
## Set options to view all columns
pd.set_option('display.max_columns', None)

In [14]:
scraper = sfc.Understat()
try:
    # Scrape the table
    out = scraper.scrape_league_table(year=2023, league='EPL')
except:
    # Catch and print any exceptions. This allows us to still close the
    # scraper below, even if an exception occurs.
    traceback.print_exc()
finally:
    # It's important to close the scraper when you're done with it. Otherwise,
    # you'll have a bunch of webdrivers open and running in the background.
    scraper.close()

d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\ScraperFC\Understat.py:361: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  columns = pd.read_html(table)[0].columns
d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\ScraperFC\Understat.py:366: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table)[0]


In [31]:
out['season'] = 2223

In [32]:
out.head()

,№,Team,M,W,D,L,G,GA,PTS,xG,NPxG,xGA,NPxGA,NPxGD,PPDA,OPPDA,DC,ODC,xPTS,season
0,1,Manchester City,38,28,5,5,94,33,89,84.32,76.71,34.21,31.92,44.78,10.33,22.32,450,154,83.04,2223
1,2,Arsenal,38,26,6,6,88,43,84,76.51,73.47,45.16,41.36,32.11,9.55,14.36,452,199,72.53,2223
2,3,Manchester United,38,23,6,9,58,43,75,71.9,69.62,50.29,48.00,21.61,11.52,13.66,391,253,66.4,2223
3,4,Newcastle United,38,19,14,5,68,33,71,76.98,72.41,41.86,38.05,34.36,9.19,10.66,376,215,71.67,2223
4,5,Liverpool,38,19,10,9,75,47,67,80.77,77.73,55.77,52.73,25.00,8.83,19.10,419,209,67.37,2223


In [33]:
understat_table = out[['Team','season', 'W', 'D', 'L','G','GA', 'xG', 'xGA','NPxG','NPxGA','PPDA',
                       'OPPDA','PTS','xPTS']]

In [34]:
# Rename "Manchester United" in Team column of understat_table to "Manchester Utd"
understat_table.loc[understat_table['Team'] == 'Manchester United', 'Team'] = 'Manchester Utd'
understat_table.loc[understat_table['Team'] == 'Newcastle United', 'Team'] = 'Newcastle Utd'
understat_table.loc[understat_table['Team'] == 'Tottenham Hotspur', 'Team'] = 'Tottenham'
understat_table.loc[understat_table['Team'] == 'Wolverhampton Wanderers', 'Team'] = 'Wolves'
understat_table.loc[understat_table['Team'] == 'Nottingham Forest', 'Team'] = "Nott'ham Forest"
understat_table.loc[understat_table['Team'] == 'Leicester', 'Team'] = 'Leicester City'
understat_table.loc[understat_table['Team'] == 'Leeds', 'Team'] = 'Leeds United'

In [35]:
understat_table.sort_values(by='Team', ascending=True, inplace=True)
understat_table.reset_index(drop=True, inplace=True)

C:\Users\ksbha\AppData\Local\Temp\ipykernel_18604\903076181.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  understat_table.sort_values(by='Team', ascending=True, inplace=True)


In [36]:
understat_table.rename(columns={'Team':'TEAM_NAME', 'season':'SEASON', 'W':'TEAM_WINS', 'D':'TEAM_DRAWS',
                                'L': 'TEAM_LOSSES', 'xPTS':'TEAM_XPTS', 'PTS':'TEAM_PTS',
                                'NPxGA':'NPXG_AGAINST', 'xGA':'XG_AGAINST'}, inplace=True)

C:\Users\ksbha\AppData\Local\Temp\ipykernel_18604\3114455065.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  understat_table.rename(columns={'Team':'TEAM_NAME', 'season':'SEASON', 'W':'TEAM_WINS', 'D':'TEAM_DRAWS',


In [37]:
fbref = sd.FBref(leagues="ENG-Premier League", seasons=["2223"])
fbref_team_standard = fbref.read_team_season_stats(stat_type='standard')
fbref_team_standard_oppo = fbref.read_team_season_stats(stat_type='standard', opponent_stats=True).reset_index()
fbref_team_shooting = fbref.read_team_season_stats(stat_type='shooting').reset_index()
fbref_team_passing = fbref.read_team_season_stats(stat_type='passing').reset_index()
fbref_team_defense = fbref.read_team_season_stats(stat_type='defense').reset_index()
fbref_team_possession = fbref.read_team_season_stats(stat_type='possession').reset_index()
fbref_team_misc = fbref.read_team_season_stats(stat_type='misc').reset_index()

[12/18/23 17:57:08] INFO     Saving cached data to C:\Users\ksbha\soccerdata\data\FBref               ]8;id=762298;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=817360;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\_common.py#91\91]8;;\

In [38]:
team_names = fbref_team_standard.reset_index().team
team_matches_played = fbref_team_standard['Playing Time'].MP
team_goals_scored = understat_table.G
team_npxG_90 = fbref_team_standard['Per 90 Minutes'].npxG
team_goals_conceded = understat_table.GA
team_xG_against_90 = fbref_team_standard_oppo['Per 90 Minutes'].xG
team_shots = fbref_team_shooting['Standard'].Sh
team_shotsOT = fbref_team_shooting['Standard'].SoT
team_pass_completed = fbref_team_passing["Total"].Cmp
team_pass_attempted = fbref_team_passing["Total"].Att
team_tackles = fbref_team_defense.Tackles.Tkl
team_tackles_won = fbref_team_defense.Tackles.TklW
team_takeons_attempted = fbref_team_possession['Take-Ons'].Att
team_takeons_completed = fbref_team_possession['Take-Ons'].Succ
team_crossesintoPA = fbref_team_passing['CrsPA']
team_fouls_against = fbref_team_misc['Performance']['Fld']
team_fouls_made = fbref_team_misc['Performance']['Fls']
team_interceptions = fbref_team_defense['Int']
team_blocks_shots = fbref_team_defense['Blocks'].Sh
team_blocks_pass = fbref_team_defense['Blocks'].Pass
team_clearances = fbref_team_defense.Clr
team_aerials_won = fbref_team_misc['Aerial Duels'].Won
team_aerials_lost = fbref_team_misc['Aerial Duels'].Lost

In [39]:
team_names = fbref_team_standard.reset_index()[['team','url']]
team_names['TEAM_FBREF_ID'] = team_names['url'].apply(lambda x: x.split('/')[3])      ## Extracting team_id from URL
team_names.drop(columns=['url'], inplace=True)
team_names.columns = team_names.columns.droplevel(1)
team_names.rename(columns={'team': 'TEAM_NAME'}, inplace=True)

In [40]:
team_names['TEAM_FBREF_ID'] = team_names['TEAM_FBREF_ID'].astype('string')

In [41]:
SNOWFLAKE_USER = 'kbharaj3'
SNOWFLAKE_PASSWORD = 'Snowfl@key0014'
SNOWFLAKE_ACCOUNT = 'qx25653.ca-central-1.aws'
SNOWFLAKE_WAREHOUSE = 'FOOTY_STORE'
SNOWFLAKE_DATABASE = 'GEGENSTATS'
SNOWFLAKE_SCHEMA = 'TABLES'

In [42]:
conn = snowflake.connector.connect(
    user=SNOWFLAKE_USER,
    password=SNOWFLAKE_PASSWORD,
    account=SNOWFLAKE_ACCOUNT,
    warehouse=SNOWFLAKE_WAREHOUSE,
    database=SNOWFLAKE_DATABASE,
    schema=SNOWFLAKE_SCHEMA
    )

[12/18/23 17:58:52] INFO     Snowflake Connector for Python Version: 3.3.1, Python Version:       ]8;id=871879;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\connection.py\connection.py]8;;\:]8;id=722531;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\connection.py#334\334]8;;\
                             3.11.5, Platform: Windows-10-10.0.22000-SP0                                           

                    INFO     This connection is in OCSP Fail Open Mode. TLS Certificates would   ]8;id=579619;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\connection.py\connection.py]8;;\:]8;id=700450;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\connection.py#1103\1103]8;;\
                             be checked for validity and revocation status. Any other                              
                             Certificate Revocation related exceptions or OCSP Responder                           
                             failures would be disregarded in favor of connectivity.                               

                    INFO     Setting use_openssl_only mode to False                              ]8;id=234593;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\connection.py\connection.py]8;;\:]8;id=447496;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\connection.py#1121\1121]8;;\

In [75]:
cur = conn.cursor()
cur.execute(f"USE WAREHOUSE {SNOWFLAKE_WAREHOUSE}")

[12/17/23 12:22:32] INFO     query: [USE WAREHOUSE FOOTY_STORE]                                       ]8;id=450257;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=272041;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=476835;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=176280;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=586777;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=500328;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

In [76]:
create_schema_sql = f"CREATE SCHEMA IF NOT EXISTS {SNOWFLAKE_SCHEMA}"
cur.execute(create_schema_sql)

[12/17/23 12:22:33] INFO     query: [CREATE SCHEMA IF NOT EXISTS TABLES]                              ]8;id=43747;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=555496;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=774915;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=313220;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=753389;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=380235;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

In [77]:
print("The Snowflake warehouse being used is:", SNOWFLAKE_WAREHOUSE)

The Snowflake warehouse being used is: FOOTY_STORE


In [78]:
create_table_query = """
CREATE TABLE IF NOT EXISTS TEAMS (
	TEAM_NAME VARCHAR(255),
	TEAM_FBREF_ID VARCHAR(255) PRIMARY KEY
);
"""

In [79]:
cur.execute(create_table_query)

[12/17/23 12:22:34] INFO     query: [CREATE TABLE IF NOT EXISTS TEAMS ( TEAM_NAME VARCHAR(255),       ]8;id=893767;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=798083;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             TEAM_FBREF_ID VARCHAR...]                                                             

                    INFO     query execution done                                                     ]8;id=852550;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=725554;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=304066;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=700914;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

In [80]:
def insert_to_snowflake_DB(primary_keys, table_name, dataframe, replace=False):
    if replace:
        cur.execute(f"TRUNCATE TABLE {table_name}")
        success, nchunks, nrows, _ = write_pandas(conn, dataframe, table_name)
    else:
        cur.execute(f"SELECT {', '.join(primary_keys)} FROM {table_name}")
        existing_ids = cur.fetchall()
        existing_ids = [id for id in existing_ids]
        dataframe.set_index(primary_keys, inplace=True)
        new_rows = dataframe[~dataframe.index.isin(existing_ids)]
        new_rows.reset_index(inplace=True)
        dataframe.reset_index(inplace=True)
        if not new_rows.empty:
            success, nchunks, nrows, _ = write_pandas(conn, new_rows, table_name)
            print(f"Inserted {nrows} new rows")
        else:
            print("No new rows to insert")

In [81]:
insert_to_snowflake_DB(['TEAM_FBREF_ID'], 'TEAMS', team_names, replace=True)

[12/17/23 12:22:35] INFO     query: [TRUNCATE TABLE TEAMS]                                            ]8;id=566371;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=449135;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=274390;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=205276;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=155024;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=832139;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

                    INFO     query: [CREATE TEMP STAGE /*                                             ]8;id=342988;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=745468;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas() */ "...]                       

                    INFO     query execution done                                                     ]8;id=951999;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=560018;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

...

[12/17/23 12:22:36] INFO     query: [PUT /* Python:snowflake.connector.pandas_tools.write_pandas() */ ]8;id=681214;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=883944;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             'file://C:\\Use...]                                                                   

                    INFO     query execution done                                                     ]8;id=652463;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=483904;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     query: [COPY INTO "TEAMS" /*                                             ]8;id=200182;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=473296;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas() */ (...]                       

[12/17/23 12:22:38] INFO     query execution done                                                     ]8;id=938219;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=483597;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=490926;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=468547;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

In [83]:
create_standard_team_stats_query = """
CREATE TABLE IF NOT EXISTS TEAM_STANDARD_STATS (
    TEAM_FBREF_ID VARCHAR(255),
    SEASON INT,
    MATCHES_PLAYED INT,
    TEAM_WINS INT,
    TEAM_DRAWS INT,
    TEAM_LOSSES INT,
    TEAM_PTS INT,
    TEAM_XPTS DOUBLE,
    PRIMARY KEY (TEAM_FBREF_ID, SEASON),
    FOREIGN KEY (TEAM_FBREF_ID) REFERENCES GEGENSTATS.TABLES.TEAMS(TEAM_FBREF_ID)
);
"""

In [84]:
cur.execute(create_standard_team_stats_query)

[12/17/23 12:22:58] INFO     query: [CREATE TABLE IF NOT EXISTS TEAM_STANDARD_STATS ( TEAM_FBREF_ID   ]8;id=26076;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=949270;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             VARCHAR(255), SEA...]                                                                 

[12/17/23 12:22:59] INFO     query execution done                                                     ]8;id=568400;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=822658;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=934180;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=595909;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

In [85]:
team_matches_played = team_matches_played.reset_index()[['team','season','MP']].rename(columns={'team': 'TEAM_NAME', 
                                                                            'season':'SEASON','MP': 'MATCHES_PLAYED'})

In [86]:
team_matches_played.SEASON = team_matches_played.SEASON.astype('int')

In [87]:
team_matches_played = team_matches_played.merge(understat_table, on=['TEAM_NAME', 'SEASON'], how='left')

In [88]:
## Merge team_matches_played with team_names to get TEAM_ID
team_matches_played = team_matches_played.merge(team_names, on='TEAM_NAME', how='left')

In [89]:
team_matches_played = team_matches_played[['TEAM_FBREF_ID', 'SEASON', 'MATCHES_PLAYED', 'TEAM_WINS', 'TEAM_DRAWS', 'TEAM_LOSSES',
                                           'TEAM_PTS', 'TEAM_XPTS']]

In [91]:
insert_to_snowflake_DB(['TEAM_FBREF_ID', 'SEASON'], 'TEAM_STANDARD_STATS', team_matches_played, replace=True)

[12/17/23 12:23:36] INFO     query: [TRUNCATE TABLE TEAM_STANDARD_STATS]                              ]8;id=25028;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=551127;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

[12/17/23 12:23:37] INFO     query execution done                                                     ]8;id=443588;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=535789;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=166597;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=318620;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

                    INFO     query: [CREATE TEMP STAGE /*                                             ]8;id=834815;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=661269;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas() */ "...]                       

                    INFO     query execution done                                                     ]8;id=511117;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=693395;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

...

                    INFO     query: [PUT /* Python:snowflake.connector.pandas_tools.write_pandas() */ ]8;id=776926;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=925242;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             'file://C:\\Use...]                                                                   

                    INFO     query execution done                                                     ]8;id=392051;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=842129;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

[12/17/23 12:23:38] INFO     query: [COPY INTO "TEAM_STANDARD_STATS" /*                               ]8;id=95229;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=154493;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write...]                                     

                    INFO     query execution done                                                     ]8;id=471450;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=683987;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=808197;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=345784;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

In [92]:
create_team_attacking_stats_query = """
CREATE TABLE IF NOT EXISTS TEAM_ATTACKING_STATS (
    TEAM_FBREF_ID VARCHAR(255),
    SEASON INT,
    GOALS_SCORED INT,
    NPXG DOUBLE,
    XG DOUBLE,
    SHOTS INT,
    SHOTS_ON_TARGET INT,
    PASS_COMPLETED INT,
    PASS_ATTEMPTED INT,
    TAKEONS_ATTEMPTED INT,
    TAKEONS_COMPLETED INT,
    CROSSES_INTO_PA INT,
    FOULS_AGAINST INT,
    PRIMARY KEY (TEAM_FBREF_ID, SEASON),
    FOREIGN KEY (TEAM_FBREF_ID) REFERENCES GEGENSTATS.TABLES.TEAMS(TEAM_FBREF_ID)
);
"""

In [93]:
cur.execute(create_team_attacking_stats_query)

[12/17/23 12:24:46] INFO     query: [CREATE TABLE IF NOT EXISTS TEAM_ATTACKING_STATS ( TEAM_FBREF_ID  ]8;id=836968;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=401232;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             VARCHAR(255), SE...]                                                                  

                    INFO     query execution done                                                     ]8;id=769378;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=404942;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=943975;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=237110;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

In [134]:
team_attacking_stats = team_npxG_90.reset_index()[['team','season','npxG']].rename(columns={'team': 'TEAM_NAME', 
                                                            'season':'SEASON','npxG': 'NPXG'})

In [135]:
team_attacking_stats.SEASON = team_attacking_stats.SEASON.astype('int')

In [136]:
team_attacking_stats = team_attacking_stats.merge(team_names, on='TEAM_NAME').merge(team_matches_played, on=['TEAM_FBREF_ID', 'SEASON'])

In [137]:
team_attacking_stats = team_attacking_stats.merge(understat_table[['TEAM_NAME','SEASON','NPxG','xG']], 
                                                  on=['TEAM_NAME','SEASON'], how='left')

In [138]:
team_attacking_stats.drop(columns=['NPXG'], inplace=True)
team_attacking_stats.rename(columns={'NPxG': 'NPXG', 'xG':'XG'}, inplace=True)

In [139]:
## Add team_goals_scored to team_attacking_stats as a new column with no merging on anything
team_attacking_stats['GOALS_SCORED'] = team_goals_scored

In [140]:
team_attacking_stats['SHOTS'] = team_shots
team_attacking_stats['SHOTS_ON_TARGET'] = team_shotsOT
team_attacking_stats['PASS_COMPLETED'] = team_pass_completed
team_attacking_stats['PASS_ATTEMPTED'] = team_pass_attempted
team_attacking_stats['TAKEONS_ATTEMPTED'] = team_takeons_attempted
team_attacking_stats['TAKEONS_COMPLETED'] = team_takeons_completed
team_attacking_stats['CROSSES_INTO_PA'] = team_crossesintoPA
team_attacking_stats['FOULS_AGAINST'] = team_fouls_against

In [141]:
team_attacking_stats.drop(columns=['TEAM_NAME'], inplace=True)

In [142]:
team_attacking_stats.drop(['MATCHES_PLAYED', 'TEAM_WINS', 'TEAM_DRAWS', 'TEAM_LOSSES','TEAM_PTS','TEAM_XPTS'], axis=1, inplace=True)

In [143]:
insert_to_snowflake_DB(['TEAM_FBREF_ID', 'SEASON'], 'TEAM_ATTACKING_STATS', team_attacking_stats, 
                       replace=True)

[12/17/23 12:36:24] INFO     query: [TRUNCATE TABLE TEAM_ATTACKING_STATS]                             ]8;id=972236;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=545331;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=711065;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=661999;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=729964;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=413657;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

                    INFO     query: [CREATE TEMP STAGE /*                                             ]8;id=855319;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=543401;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas() */ "...]                       

                    INFO     query execution done                                                     ]8;id=15114;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=204692;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

...

                    INFO     query: [PUT /* Python:snowflake.connector.pandas_tools.write_pandas() */ ]8;id=233407;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=884754;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             'file://C:\\Use...]                                                                   

                    INFO     query execution done                                                     ]8;id=732350;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=718547;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

[12/17/23 12:36:25] INFO     query: [COPY INTO "TEAM_ATTACKING_STATS" /*                              ]8;id=78854;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=752805;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.writ...]                                      

[12/17/23 12:36:26] INFO     query execution done                                                     ]8;id=662132;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=728745;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=877397;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=403105;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

In [167]:
create_team_defending_stats_query = """
CREATE TABLE IF NOT EXISTS TEAM_DEFENDING_STATS (
    TEAM_FBREF_ID VARCHAR(255),
    SEASON INT,
    GOALS_CONCEDED INT,
    XG_AGAINST DOUBLE,
    NPXG_AGAINST DOUBLE,
    TACKLES INT,
    TACKLES_WON INT,
    FOULS_MADE INT,
    INTERCEPTIONS INT,
    BLOCKED_SHOTS INT,
    BLOCKED_PASSES INT,
    CLEARANCES INT,
    PPDA DOUBLE,
    PRIMARY KEY (TEAM_FBREF_ID, SEASON),
    FOREIGN KEY (TEAM_FBREF_ID) REFERENCES GEGENSTATS.TABLES.TEAMS(TEAM_FBREF_ID)
);
"""

In [168]:
cur.execute(create_team_defending_stats_query)

[12/17/23 12:50:29] INFO     query: [CREATE TABLE IF NOT EXISTS TEAM_DEFENDING_STATS ( TEAM_FBREF_ID  ]8;id=459876;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=879410;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             VARCHAR(255), SE...]                                                                  

                    INFO     query execution done                                                     ]8;id=450150;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=89739;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=904242;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=496376;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

In [169]:
team_defending_stats = team_names.copy()

In [170]:
team_defending_stats['SEASON'] = team_attacking_stats.SEASON
team_defending_stats['GOALS_CONCEDED'] = team_goals_conceded
team_defending_stats['XG_AGAINST'] = understat_table['XG_AGAINST']
team_defending_stats['NPXG_AGAINST'] = understat_table['NPXG_AGAINST']
team_defending_stats['TACKLES'] = team_tackles
team_defending_stats['TACKLES_WON'] = team_tackles_won
team_defending_stats['FOULS_MADE'] = team_fouls_made
team_defending_stats['INTERCEPTIONS'] = team_interceptions
team_defending_stats['BLOCKED_SHOTS'] = team_blocks_shots
team_defending_stats['BLOCKED_PASSES'] = team_blocks_pass
team_defending_stats['CLEARANCES'] = team_clearances
team_defending_stats['PPDA'] = understat_table['PPDA']

In [171]:
team_defending_stats.drop(columns=['TEAM_NAME'], inplace=True)

In [172]:
insert_to_snowflake_DB(['TEAM_FBREF_ID', 'SEASON'], 'TEAM_DEFENDING_STATS', team_defending_stats,
                       replace=True)

[12/17/23 12:50:37] INFO     query: [TRUNCATE TABLE TEAM_DEFENDING_STATS]                             ]8;id=102656;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=781939;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=817091;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=962841;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=317256;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=29203;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

                    INFO     query: [CREATE TEMP STAGE /*                                             ]8;id=394475;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=629001;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas() */ "...]                       

                    INFO     query execution done                                                     ]8;id=329290;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=319445;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

...

                    INFO     query: [PUT /* Python:snowflake.connector.pandas_tools.write_pandas() */ ]8;id=56019;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=951922;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             'file://C:\\Use...]                                                                   

                    INFO     query execution done                                                     ]8;id=671122;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=373023;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

[12/17/23 12:50:38] INFO     query: [COPY INTO "TEAM_DEFENDING_STATS" /*                              ]8;id=943106;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=803002;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.writ...]                                      

[12/17/23 12:50:39] INFO     query execution done                                                     ]8;id=943037;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=132889;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=249250;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=108987;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

In [44]:
create_team_misc_stats_query = """
CREATE TABLE IF NOT EXISTS TEAM_MISC_STATS (
    TEAM_FBREF_ID VARCHAR(255),
    SEASON INT,
    AERIALS_WON INT,
    AERIALS_LOST INT,
    PRIMARY KEY (TEAM_FBREF_ID, SEASON),
    FOREIGN KEY (TEAM_FBREF_ID) REFERENCES GEGENSTATS.TABLES.TEAMS(TEAM_FBREF_ID)
);
"""

In [45]:
cur.execute(create_team_misc_stats_query)

[12/16/23 19:28:08] INFO     query: [CREATE TABLE IF NOT EXISTS TEAM_MISC_STATS ( TEAM_FBREF_ID       ]8;id=443202;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=400331;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             VARCHAR(255), SEASON ...]                                                             

                    INFO     query execution done                                                     ]8;id=666840;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=682396;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=943051;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=220151;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

In [46]:
team_misc_stats = team_names.copy()

In [47]:
team_misc_stats['SEASON'] = team_attacking_stats.SEASON
team_misc_stats['AERIALS_WON'] = team_aerials_won
team_misc_stats['AERIALS_LOST'] = team_aerials_lost

In [48]:
team_misc_stats.drop(columns=['TEAM_NAME'], inplace=True)

In [49]:
insert_to_snowflake_DB(['TEAM_FBREF_ID', 'SEASON'], 'TEAM_MISC_STATS', team_misc_stats, replace=True)

[12/16/23 19:28:13] INFO     query: [TRUNCATE TABLE TEAM_MISC_STATS]                                  ]8;id=510241;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=648190;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=838110;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=616546;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=340239;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=47730;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

                    INFO     query: [CREATE TEMP STAGE /*                                             ]8;id=571380;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=905448;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas() */ "...]                       

                    INFO     query execution done                                                     ]8;id=116818;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=556184;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

...

                    INFO     query: [PUT /* Python:snowflake.connector.pandas_tools.write_pandas() */ ]8;id=568504;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=673980;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             'file://C:\\Use...]                                                                   

                    INFO     query execution done                                                     ]8;id=746917;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=938869;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

[12/16/23 19:28:14] INFO     query: [COPY INTO "TEAM_MISC_STATS" /*                                   ]8;id=890866;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=145512;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pan...]                                 

[12/16/23 19:28:15] INFO     query execution done                                                     ]8;id=791725;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=340072;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=297716;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=329762;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

# Now, inclusion of Match and Player Data (TBC!)

In [50]:
fbref_schedule = fbref.read_schedule()

In [74]:
create_competition_query = """
CREATE TABLE IF NOT EXISTS COMPETITIONS (
	COMPETITION VARCHAR(255) PRIMARY KEY,
    COMPETITION_NAME VARCHAR(255),
    SEASON INT
);
"""

In [75]:
cur.execute(create_competition_query)

[12/16/23 21:26:35] INFO     query: [CREATE TABLE IF NOT EXISTS COMPETITIONS ( COMPETITION            ]8;id=72708;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=646744;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             VARCHAR(255) PRIMARY KEY, ...]                                                        

                    INFO     query execution done                                                     ]8;id=417349;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=368001;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=942967;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=671501;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

In [76]:
df_competitions = pd.DataFrame({'COMPETITION': ['ENG-Premier League-2223'], 'COMPETITION_NAME': ['English Premier League'], 'SEASON': [2223]})

In [77]:
insert_to_snowflake_DB(['COMPETITION'], 'COMPETITIONS', df_competitions, replace=True)

[12/16/23 21:26:50] INFO     query: [TRUNCATE TABLE COMPETITIONS]                                     ]8;id=248502;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=891540;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=259520;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=139240;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=607407;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=251036;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

                    INFO     query: [CREATE TEMP STAGE /*                                             ]8;id=734492;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=470203;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas() */ "...]                       

                    INFO     query execution done                                                     ]8;id=613330;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=368048;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

...

                    INFO     query: [PUT /* Python:snowflake.connector.pandas_tools.write_pandas() */ ]8;id=276511;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=987815;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             'file://C:\\Use...]                                                                   

[12/16/23 21:26:51] INFO     query execution done                                                     ]8;id=88156;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=689889;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     query: [COPY INTO "COMPETITIONS" /*                                      ]8;id=426211;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=53798;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas...]                              

[12/16/23 21:26:52] INFO     query execution done                                                     ]8;id=905759;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=658536;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=745307;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=651951;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

In [70]:
create_season_query = """
CREATE TABLE IF NOT EXISTS SEASONS (
	SEASON INT PRIMARY KEY,
    START_YEAR INT,
    END_YEAR INT
);
"""

In [71]:
cur.execute(create_season_query)

[12/16/23 21:22:31] INFO     query: [CREATE TABLE IF NOT EXISTS SEASONS ( SEASON INT PRIMARY KEY,     ]8;id=43246;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=532362;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             START_YEAR INT, END...]                                                               

                    INFO     query execution done                                                     ]8;id=532714;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=29953;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=61955;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=770343;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

In [72]:
df_seasons = pd.DataFrame({'SEASON': [2223], 'START_YEAR': 2022, 'END_YEAR': [2023]})

In [73]:
insert_to_snowflake_DB(['SEASON'], 'SEASONS', df_seasons, replace=True)

[12/16/23 21:25:09] INFO     query: [TRUNCATE TABLE SEASONS]                                          ]8;id=243293;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=816180;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=640387;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=126103;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=454518;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=525809;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

                    INFO     query: [CREATE TEMP STAGE /*                                             ]8;id=88545;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=123372;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas() */ "...]                       

                    INFO     query execution done                                                     ]8;id=826751;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=746096;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

...

                    INFO     query: [PUT /* Python:snowflake.connector.pandas_tools.write_pandas() */ ]8;id=264986;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=922777;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             'file://C:\\Use...]                                                                   

[12/16/23 21:25:10] INFO     query execution done                                                     ]8;id=636144;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=687055;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     query: [COPY INTO "SEASONS" /*                                           ]8;id=418610;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=190362;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas() */...]                         

[12/16/23 21:25:11] INFO     query execution done                                                     ]8;id=335034;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=35611;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=520419;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=441039;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

In [80]:
create_stadium_query = """
CREATE TABLE IF NOT EXISTS STADIUMS (
	STADIUM VARCHAR(255) PRIMARY KEY,
    TEAM_FBREF_ID VARCHAR(255)
);
"""

In [81]:
cur.execute(create_stadium_query)

[12/16/23 21:31:02] INFO     query: [CREATE TABLE IF NOT EXISTS STADIUMS ( STADIUM VARCHAR(255)       ]8;id=457364;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=484705;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             PRIMARY KEY, TEAM_FBR...]                                                             

[12/16/23 21:31:03] INFO     query execution done                                                     ]8;id=322774;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=144197;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=711143;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=166371;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

In [92]:
df_stadiums = fbref_schedule.reset_index()[['home_team', 
            'venue']].drop_duplicates().rename(columns={'home_team': 'TEAM_NAME', 'venue': 'STADIUM'})

In [97]:
df_stadiums = df_stadiums.merge(team_names, on='TEAM_NAME')[['STADIUM', 'TEAM_FBREF_ID']]

In [98]:
insert_to_snowflake_DB(['STADIUM'], 'STADIUMS', df_stadiums, replace=True)

[12/16/23 21:35:06] INFO     query: [TRUNCATE TABLE STADIUMS]                                         ]8;id=372918;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=597315;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

[12/16/23 21:35:07] INFO     query execution done                                                     ]8;id=988229;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=363610;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=69819;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=136686;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

                    INFO     query: [CREATE TEMP STAGE /*                                             ]8;id=917438;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=116844;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas() */ "...]                       

                    INFO     query execution done                                                     ]8;id=44807;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=926951;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

...

                    INFO     query: [PUT /* Python:snowflake.connector.pandas_tools.write_pandas() */ ]8;id=619629;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=714654;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             'file://C:\\Use...]                                                                   

                    INFO     query execution done                                                     ]8;id=668422;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=828231;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

[12/16/23 21:35:08] INFO     query: [COPY INTO "STADIUMS" /*                                          ]8;id=533023;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=575052;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas() *...]                          

[12/16/23 21:35:09] INFO     query execution done                                                     ]8;id=684164;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=322856;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=614995;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=128242;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

In [195]:
create_match_query = """
CREATE TABLE IF NOT EXISTS MATCHES (
    MATCH_ID VARCHAR(255) PRIMARY KEY,
    DATE_TIME VARCHAR(255), 
    HOME_TEAM_ID VARCHAR(255),
    AWAY_TEAM_ID VARCHAR(255),
    COMPETITION VARCHAR(255),
    SEASON INT,
    STADIUM VARCHAR(255),
    GAMEWEEK INT,
    DAY VARCHAR(255),
    HOME_TEAM_SCORE INT,
    AWAY_TEAM_SCORE INT,
    HOME_TEAM_XG DOUBLE,
    AWAY_TEAM_XG DOUBLE,
    ATTENDANCE INT,
    REFEREE VARCHAR(255),
    FOREIGN KEY (HOME_TEAM_ID) REFERENCES GEGENSTATS.TABLES.TEAMS(TEAM_FBREF_ID),
    FOREIGN KEY (AWAY_TEAM_ID) REFERENCES GEGENSTATS.TABLES.TEAMS(TEAM_FBREF_ID),
    FOREIGN KEY (COMPETITION) REFERENCES GEGENSTATS.TABLES.COMPETITIONS(COMPETITION),
    FOREIGN KEY (SEASON) REFERENCES GEGENSTATS.TABLES.SEASONS(SEASON),
    FOREIGN KEY (STADIUM) REFERENCES GEGENSTATS.TABLES.STADIUMS(STADIUM)
);
"""

In [196]:
cur.execute(create_match_query)

[12/16/23 23:47:53] INFO     query: [CREATE TABLE IF NOT EXISTS MATCHES ( MATCH_ID VARCHAR(255)       ]8;id=596840;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=187401;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             PRIMARY KEY, DATE_TIM...]                                                             

[12/16/23 23:47:54] INFO     query execution done                                                     ]8;id=155119;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=499811;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=384936;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=118883;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

In [54]:
fbref_lineups = fbref.read_lineup()

[12/16/23 20:07:13] INFO     [1/380] Retrieving game with id=e62f6e78                                  ]8;id=962105;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py\fbref.py]8;;\:]8;id=32060;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py#878\878]8;;\

d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py:893: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_table.loc[:bench_idx, "is_starter"] = True
d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py:893: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_table.loc[:bench_idx, "is_starter"] = True


                    INFO     [2/380] Retrieving game with id=877e3193                                  ]8;id=731678;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py\fbref.py]8;;\:]8;id=284455;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py#878\878]8;;\

d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py:893: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_table.loc[:bench_idx, "is_starter"] = True
d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py:893: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_table.loc[:bench_idx, "is_starter"] = True


                    INFO     [3/380] Retrieving game with id=3a917cee                                  ]8;id=392538;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py\fbref.py]8;;\:]8;id=845603;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py#878\878]8;;\

...

d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py:893: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_table.loc[:bench_idx, "is_starter"] = True
d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py:893: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_table.loc[:bench_idx, "is_starter"] = True


[12/16/23 20:35:47] INFO     [379/380] Retrieving game with id=64823dc9                                ]8;id=794785;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py\fbref.py]8;;\:]8;id=479672;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py#878\878]8;;\

d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py:893: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_table.loc[:bench_idx, "is_starter"] = True
d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py:893: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_table.loc[:bench_idx, "is_starter"] = True


[12/16/23 20:35:51] INFO     [380/380] Retrieving game with id=8e5c6ea7                                ]8;id=236926;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py\fbref.py]8;;\:]8;id=246376;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py#878\878]8;;\

d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py:893: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_table.loc[:bench_idx, "is_starter"] = True
d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py:893: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_table.loc[:bench_idx, "is_starter"] = True


In [197]:
df_matches = fbref_schedule.reset_index()[['game_id', 'date', 'time', 'home_team', 'away_team', 'league', 'season', 'venue',
                                           'week', 'day', 'score', 'home_xg', 'away_xg', 'attendance', 'referee']]

In [198]:
df_matches = df_matches.merge(team_names.rename(columns={'TEAM_FBREF_ID': 'home_team_id', 
                                                         'TEAM_NAME': 'home_team'}), on='home_team', how='left')

In [199]:
df_matches = df_matches.merge(team_names.rename(columns={'TEAM_FBREF_ID': 'away_team_id', 
                                                         'TEAM_NAME': 'away_team'}), on='away_team', how='left')

In [200]:
# Combine df_matches date and time to a dt.datatime configuration
df_matches['date'] = df_matches['date'].astype(str)
df_matches['time'] = df_matches['time'].astype(str)
df_matches['date_time'] = df_matches['date'] + ' ' + df_matches['time']
df_matches['date_time'] = pd.to_datetime(df_matches['date_time'])

In [201]:
# Split score in df_matches by "-" to home_team_score and away_team_score 
df_matches[['home_team_score', 'away_team_score']] = df_matches['score'].str.split('–', expand=True)

In [202]:
df_matches.rename(columns={'game_id': 'MATCH_ID', 'date_time': 'DATE_TIME', 'home_team_id': 'HOME_TEAM_ID', 'away_team_id': 'AWAY_TEAM_ID',
                           'league': 'COMPETITION','season' : 'SEASON','venue': 'STADIUM', 'week': 'GAMEWEEK', 'day': 'DAY',
                           'home_team_score': 'HOME_TEAM_SCORE', 'away_team_score':'AWAY_TEAM_SCORE', 'home_xg':'HOME_TEAM_XG',
                            'away_xg':'AWAY_TEAM_XG','attendance':'ATTENDANCE', 'referee': 'REFEREE'}, inplace=True)

In [203]:
df_matches.drop(columns=['date', 'time', 'home_team', 'away_team', 'score'], inplace=True)

In [209]:
df_matches['DATE_TIME'] = df_matches['DATE_TIME'].astype('string')

In [210]:
insert_to_snowflake_DB(['MATCH_ID'], 'MATCHES', df_matches, replace=True)

[12/16/23 23:50:10] INFO     query: [TRUNCATE TABLE MATCHES]                                          ]8;id=495017;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=330007;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=896305;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=738209;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=660296;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=752224;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

                    INFO     query: [CREATE TEMP STAGE /*                                             ]8;id=97878;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=742150;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas() */ "...]                       

                    INFO     query execution done                                                     ]8;id=647391;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=240241;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

...

                    INFO     query: [PUT /* Python:snowflake.connector.pandas_tools.write_pandas() */ ]8;id=239288;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=21443;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             'file://C:\\Use...]                                                                   

[12/16/23 23:50:11] INFO     query execution done                                                     ]8;id=970462;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=848516;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     query: [COPY INTO "MATCHES" /*                                           ]8;id=215190;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=405739;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas() */...]                         

[12/16/23 23:50:12] INFO     query execution done                                                     ]8;id=283616;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=683608;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=516149;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=716320;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

0      2022-08-05 20:00:00
1      2022-08-06 15:00:00
2      2022-08-06 17:30:00
3      2022-08-06 12:30:00
4      2022-08-06 15:00:00
              ...         
375    2023-05-28 16:30:00
376    2023-05-28 16:30:00
377    2023-05-28 16:30:00
378    2023-05-28 16:30:00
379    2023-05-28 16:30:00
Name: DATE_TIME, Length: 380, dtype: string

### TBD- Adding shot events data!

In [139]:
create_shots_query = """
CREATE TABLE IF NOT EXISTS SHOT_EVENTS (
    MATCH_ID VARCHAR(255) PRIMARY KEY,
    SEASON INT,
    TEAM_FBREF_ID VARCHAR(255),
    PLAYER_FBREF_NAME VARCHAR(255),
    MINUTE INT,
    XG DOUBLE,
    PSXG DOUBLE,
    OUTCOME VARCHAR(255),
    DISTANCE INT,
    BODY_PART VARCHAR(255),
    NOTES VARCHAR(255),
    SCA_1_PLAYER_FBREF_NAME VARCHAR(255),
    SCA_1_PLAYER_FBREF_EVENT VARCHAR(255),
    SCA_2_PLAYER_FBREF_NAME VARCHAR(255),
    SCA_2_PLAYER_FBREF_EVENT VARCHAR(255)
);
"""

In [134]:
fbref_shot_events = fbref.read_shot_events()

[12/16/23 22:48:45] INFO     [1/380] Retrieving game with id=e62f6e78                                 ]8;id=408563;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py\fbref.py]8;;\:]8;id=635615;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py#1057\1057]8;;\

                    INFO     [2/380] Retrieving game with id=877e3193                                 ]8;id=347372;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py\fbref.py]8;;\:]8;id=173378;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py#1057\1057]8;;\

                    INFO     [3/380] Retrieving game with id=3a917cee                                 ]8;id=889516;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py\fbref.py]8;;\:]8;id=935883;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py#1057\1057]8;;\

                    INFO     [4/380] Retrieving game with id=6713c1dc                                 ]8;id=422286;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py\fbref.py]8;;\:]8;id=982902;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py#1057\1057]8;;\

                    INFO     [5/380] Retrieving game with id=82702941                                 ]8;id=658050;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py\fbref.py]8;;\:]8;id=394343;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py#1057\1057]8;;\

...

                    INFO     [376/380] Retrieving game with id=94de848f                               ]8;id=425962;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py\fbref.py]8;;\:]8;id=399719;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py#1057\1057]8;;\

                    INFO     [377/380] Retrieving game with id=c9c73ddd                               ]8;id=388954;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py\fbref.py]8;;\:]8;id=378588;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py#1057\1057]8;;\

                    INFO     [378/380] Retrieving game with id=a96c9915                               ]8;id=817430;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py\fbref.py]8;;\:]8;id=817154;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py#1057\1057]8;;\

                    INFO     [379/380] Retrieving game with id=64823dc9                               ]8;id=547458;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py\fbref.py]8;;\:]8;id=184681;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py#1057\1057]8;;\

                    INFO     [380/380] Retrieving game with id=8e5c6ea7                               ]8;id=25493;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py\fbref.py]8;;\:]8;id=974864;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py#1057\1057]8;;\

In [145]:
fbref_schedule.reset_index()[['game', 'game_id']]

,game,game_id
0,2022-08-05 Crystal Palace-Arsenal,e62f6e78
1,2022-08-06 Bournemouth-Aston Villa,877e3193
2,2022-08-06 Everton-Chelsea,3a917cee
3,2022-08-06 Fulham-Liverpool,6713c1dc
4,2022-08-06 Leeds United-Wolves,82702941
...,...,...
375,2023-05-28 Everton-Bournemouth,94de848f
376,2023-05-28 Leeds United-Tottenham,c9c73ddd
377,2023-05-28 Leicester City-West Ham,a96c9915
378,2023-05-28 Manchester Utd-Fulham,64823dc9


In [149]:
df_shots = fbref_shot_events.reset_index().copy()

In [ ]:
df_shots.columns = ['_'.join(col).rstrip('_') for col in df_shots.columns.values]

In [162]:
df_shots.merge(fbref_schedule.reset_index()[['game', 'game_id']], on='game', how='left')

,league,season,game,minute,player,team,xG,PSxG,outcome,distance,body_part,notes,SCA 1_player,SCA 1_event,SCA 2_player,SCA 2_event,game_id
0,ENG-Premier League,2223,2022-08-05 Crystal Palace-Arsenal,4,Gabriel Jesus,Arsenal,0.15,<NA>,Blocked,14,Right Foot,<NA>,Gabriel Jesus,Take-On,Gabriel Jesus,Take-On,e62f6e78
1,ENG-Premier League,2223,2022-08-05 Crystal Palace-Arsenal,4,Gabriel Martinelli,Arsenal,0.22,<NA>,Off Target,8,Left Foot,<NA>,Gabriel Jesus,Shot,<NA>,<NA>,e62f6e78
2,ENG-Premier League,2223,2022-08-05 Crystal Palace-Arsenal,8,Oleksandr Zinchenko,Arsenal,0.03,0.06,Saved,18,Left Foot,Deflected,Granit Xhaka,Pass (Live),Gabriel Jesus,Pass (Live),e62f6e78
3,ENG-Premier League,2223,2022-08-05 Crystal Palace-Arsenal,9,Thomas Partey,Arsenal,0.1,<NA>,Blocked,21,Right Foot,<NA>,Oleksandr Zinchenko,Pass (Live),Granit Xhaka,Pass (Live),e62f6e78
4,ENG-Premier League,2223,2022-08-05 Crystal Palace-Arsenal,20,Gabriel Martinelli,Arsenal,0.16,0.29,Goal,5,Head,<NA>,Oleksandr Zinchenko,Pass (Live),Bukayo Saka,Pass (Dead),e62f6e78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9604,ENG-Premier League,2223,2023-05-28 Southampton-Liverpool,90+2,Mohamed Salah,Liverpool,0.17,<NA>,Off Target,13,Left Foot,NaN,Harvey Elliott,Pass (Live),Jordan Henderson,Pass (Live),8e5c6ea7
9605,ENG-Premier League,2223,2023-05-28 Southampton-Liverpool,90+5,Kyle Walker-Peters,Southampton,0.04,0.23,Saved,18,Right Foot,NaN,Kyle Walker-Peters,Take-On,Samuel Amo-Ameyaw,Pass (Live),8e5c6ea7
9606,ENG-Premier League,2223,2023-05-28 Southampton-Liverpool,90+5,Jan Bednarek,Southampton,0.05,<NA>,Off Target,8,Right Foot,NaN,Stuart Armstrong,Pass (Dead),Kyle Walker-Peters,Shot,8e5c6ea7
9607,ENG-Premier League,2223,2023-05-28 Southampton-Liverpool,90+7,Mohamed Salah,Liverpool,0.27,<NA>,Off Target,6,Left Foot,NaN,Mohamed Salah,Take-On,Trent Alexander-Arnold,Pass (Live),8e5c6ea7
